In this notebook we will work on a prediction problem of time series data. 

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn

The file was downloaded from http://archive.ics.uci.edu/ml/machine-learning-databases/00235/  
  
The original date has seperated date and time columns, and it would have taken too much time to combine into a datetime after reading. So, we parse them right here.

In [ ]:
dt_parser = lambda x, y: pd.datetime.strptime(str(x) + str(y), '%d/%m/%Y%H:%M:%S')
df = pd.read_csv('../input/householdpowerconsumption/household_power_consumption.txt', sep = ';',
                parse_dates={'dt' : ['Date', 'Time']}, date_parser = dt_parser, index_col='dt',
                low_memory=False)
df.head()

In [ ]:
df.info()

* We can see that we have over 2 million rows, so resampling might be needed. 
* Most of the features needs type cast also.

In [ ]:
for col in df:
    print(df[col].value_counts(dropna=False))

We can see that there are 25979 '?'s in the first several columns and an equal amount of "NaN" in the last. Considering the number doesn't make a huge portion in over 2 million, we replace them with mean. 

In [ ]:
df.replace('?', np.nan, inplace=True)
for col in df:
    df[col] = df[col].astype(float)
df.fillna(df.mean(), inplace=True)

In [ ]:
# Check
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
def plot_mean_std(data):
    plt.style.use('seaborn')
    fig, (ax1, ax2) = plt.subplots(2,1, figsize=(10,10))
    ax1.plot(data.index.to_pydatetime(), data['mean'], color= 'b')
    ax2.plot(data.index.to_pydatetime(), data['std'], color= 'g')
    

In [ ]:
r = df['Global_active_power'].resample('D').agg(['mean', 'std'])
plot_mean_std(r)

Right off the bat, we can tell that there is probably periodicity involved in the data, and the period is a year. To reflect such periodicity over long time, LSTM might be useful.

In [ ]:
r = df['Global_reactive_power'].resample('D').agg(['mean', 'std'])
plot_mean_std(r)

In [ ]:
r = df['Global_intensity'].resample('D').agg(['mean', 'std'])
plot_mean_std(r)

So some periodicity in both mean and std of global intensity is also seen. 

In [ ]:
# resample by day too messy. 
sub1 = df['Sub_metering_1'].resample('2W').agg(['mean', 'std'])
plot_mean_std(sub1)

In [ ]:
sub2 = df['Sub_metering_2'].resample('2W').agg(['mean', 'std'])
plot_mean_std(sub2)

In [ ]:
sub3 = df['Sub_metering_3'].resample('2W').agg(['mean', 'std'])
plot_mean_std(sub3)

The up and down trends that starts and ends at July each year are also pretty obvious. But the pattern seems to differ each year for submeterings.  

In [ ]:
volt = df['Voltage'].resample('d').agg(['mean', 'std'])
plot_mean_std(volt)

Voltage doesn't variate by a lot.

Need to scale the data first as they're in quite different ranges.

In [ ]:
# Resample because of the high computational load.
df = df.resample('h').mean()
print(df.shape)

In [ ]:
values = df.values
scaler = MinMaxScaler(feature_range=(0, 1))
df[:] = scaler.fit_transform(values)
df.head()

In this task, we can define our objective as to predict the global active power of the current timestep, on the basis of the informations of the other 6 variables, and the global active power of the previous timesteps. 

Initially used only one step history. Now we try to include longer history as training data, see if it improves performance.

In [ ]:
prev_steps = 10

for i in range(prev_steps):
    # t - (i+1), i+1 = 1, ..., prev_steps
    df0 = df.iloc[:,:7].shift(1)
    if (i!=0):
        df0.rename(columns = {x:x[:-3] +'-%02d' % (i+1) for x in df0.columns}, inplace=True)
    else:
        df0.rename(columns = {x:x +'-%02d'% (i+1) for x in df0.columns}, inplace=True)
        
    df = pd.concat([df0,df], axis = 1)

# t+1
df['GAP next'] = df['Global_active_power'].shift(-1)    
df.dropna(inplace = True)
df.head()

Let's use all data except for the last year as training set, and the last year as the test set.

In [ ]:
train = df[:'2009-07-01 00:00:00'].values 
test = df['2009-07-01 00:00:00':].values
print(train.shape, test.shape)

In [ ]:
train_X, train_y = train[:,:-1], train[:,-1]
test_X, test_y = test[:,:-1], test[:,-1]

In [ ]:
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

In [ ]:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.metrics import mean_squared_error

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dropout(0.4))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

In [ ]:
model.fit(train_X, train_y, epochs=250, batch_size=128, validation_data=(test_X, test_y), shuffle=False)
y_hat = model.predict(test_X)

In [ ]:
test_X.shape

In [ ]:
# Get back only the last 7 columns for inversion. 
test_X = test_X[:,:,-7:].reshape((test_X.shape[0], 7))

# invert scaling for y_hat
y_hat_x = np.concatenate((y_hat, test_X[:, 1:]), axis=1)
y_hat_x = scaler.inverse_transform(y_hat_x)
inv_y_hat = y_hat_x[:,0]

# invert scaling for the original test value
test_y = test_y.reshape((test_y.shape[0], 1))
test_y_x = np.concatenate((test_y, test_X[:, 1:]), axis=1)
test_y_x = scaler.inverse_transform(test_y_x)
inv_y = test_y_x[:,0]
# calculate RMSE
rmse = np.sqrt(mean_squared_error(inv_y, inv_y_hat))
print('Test RMSE: %.3f' % rmse)

In [ ]:
x = range(inv_y.shape[0])
plt.plot(x, inv_y, label="actual")
plt.plot(x, inv_y_hat, label="prediction")
plt.ylabel('Global_active_power', size = 15)
plt.xlabel('Time step', size = 15)
plt.legend(fontsize=15)
plt.show()

As I increase the number of epochs and change other hyperparameters (without using grid search), the result matches marginally better. If more history is included, the performance is also slightly improved. Somehow the prediction are better at the beginning and the end phase of the period, but underpredicts in the period. 